## Tutorial of Interventions on Non-transformer Model: MLPs

In [1]:
__author__ = "Zhengxuan Wu"
__version__ = "12/20/2023"

### Overview

This tutorials show how to use this library on non-transformer models, such as MLPs. The set-ups are pretty much the same as standard transformer-based models.

### Set-up

In [2]:
try:
    # This library is our indicator that the required installs
    # need to be done.
    import pyvene

except ModuleNotFoundError:
    !pip install git+https://github.com/frankaging/pyvene.git

[2024-01-11 01:32:46,791] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
import torch
import pandas as pd
from pyvene import embed_to_distrib, top_vals, format_token
from pyvene import (
    IntervenableModel,
    VanillaIntervention,
    RotatedSpaceIntervention,
    LowRankRotatedSpaceIntervention,
    IntervenableRepresentationConfig,
    IntervenableConfig,
)
from pyvene.models.mlp.modelings_mlp import MLPConfig
from pyvene import create_mlp_classifier

%config InlineBackend.figure_formats = ['svg']
from plotnine import (
    ggplot,
    geom_tile,
    aes,
    facet_wrap,
    theme,
    element_text,
    geom_bar,
    geom_hline,
    scale_y_log10,
)

config, tokenizer, mlp = create_mlp_classifier(MLPConfig(h_dim=32, n_layer=1))

loaded model


### Intervene in middle layer by partitioning representations into subspaces

MLP layer may contain only a single "token" representation each layer. As a result, we often want to intervene on a subspace of this "token" representation to localize a concept.

In [4]:
intervenable_config = IntervenableConfig(
    intervenable_model_type=type(mlp),
    intervenable_representations=[
        IntervenableRepresentationConfig(
            0,
            "block_output",
            "pos",  # mlp layer creates a single token reprs
            1,
            subspace_partition=[
                [0, 16],
                [16, 32],
            ],  # partition into two sets of subspaces
        ),
    ],
    intervenable_interventions_type=RotatedSpaceIntervention,
)
intervenable = IntervenableModel(intervenable_config, mlp)

base = {"inputs_embeds": torch.rand(1, 1, 32)}
source = {"inputs_embeds": torch.rand(1, 1, 32)}
print("base", intervenable(base))
print("source", intervenable(source))

base ((tensor([[-0.1626, -0.1709]]),), None)
source ((tensor([[-0.1141, -0.1481]]),), None)


In [5]:
_, counterfactual_outputs = intervenable(
    base, [source], {"sources->base": ([[[0]]], [[[0]]])}, subspaces=[[[1, 0]]]
)

In [6]:
counterfactual_outputs  # this should be the same as source.

(tensor([[-0.1141, -0.1481]], grad_fn=<SqueezeBackward1>),)

### Intervene the subspace with multiple sources

In [7]:
intervenable_config = IntervenableConfig(
    intervenable_model_type=type(mlp),
    intervenable_representations=[
        IntervenableRepresentationConfig(
            0,
            "block_output",
            "pos",  # mlp layer creates a single token reprs
            1,
            intervenable_low_rank_dimension=32,
            subspace_partition=[
                [0, 16],
                [16, 32],
            ],  # partition into two sets of subspaces
            intervention_link_key=0,  # linked ones target the same subspace
        ),
        IntervenableRepresentationConfig(
            0,
            "block_output",
            "pos",  # mlp layer creates a single token reprs
            1,
            intervenable_low_rank_dimension=32,
            subspace_partition=[
                [0, 16],
                [16, 32],
            ],  # partition into two sets of subspaces
            intervention_link_key=0,  # linked ones target the same subspace
        ),
    ],
    intervenable_interventions_type=LowRankRotatedSpaceIntervention,
)
intervenable = IntervenableModel(intervenable_config, mlp)

base = {"inputs_embeds": torch.rand(10, 1, 32)}
source = {"inputs_embeds": torch.rand(10, 1, 32)}
print("base", intervenable(base))
print("source", intervenable(source))

base ((tensor([[-0.0987, -0.1492],
        [-0.0989, -0.0679],
        [-0.1424, -0.0600],
        [-0.1842, -0.0832],
        [-0.1593, -0.1935],
        [-0.0948, -0.0782],
        [-0.1486, -0.1780],
        [-0.1060, -0.1453],
        [-0.1249, -0.2012],
        [-0.1365, -0.1093]]),), None)
source ((tensor([[-0.1486, -0.2303],
        [-0.0673, -0.1547],
        [-0.1168, -0.0861],
        [-0.1526, -0.1426],
        [-0.1433, -0.1334],
        [-0.1238, -0.2006],
        [-0.1416, -0.1591],
        [-0.1560, -0.0976],
        [-0.1467, -0.1371],
        [-0.0933, -0.2142]]),), None)


In [8]:
_, counterfactual_outputs = intervenable(
    base,
    [source, source],
    {"sources->base": ([[[0]] * 10, [[0]] * 10], [[[0]] * 10, [[0]] * 10])},
    subspaces=[[[1]] * 10, [[0]] * 10],
)
print(counterfactual_outputs)  # this should be the same as the source output
counterfactual_outputs[
    0
].sum().backward()  # fake call to make sure gradient can be populated

(tensor([[-0.1486, -0.2303],
        [-0.0673, -0.1547],
        [-0.1168, -0.0861],
        [-0.1526, -0.1426],
        [-0.1433, -0.1334],
        [-0.1238, -0.2006],
        [-0.1416, -0.1591],
        [-0.1560, -0.0976],
        [-0.1467, -0.1371],
        [-0.0933, -0.2142]], grad_fn=<SqueezeBackward1>),)
